In [1]:
##load the libraries
import pandas as pd
from django.db.models import Q
##load the django model
from twitterSentiment.models import Company, CompanyFinancials, TwitterText, CompanyAltmanZscore, CompanySentiment

In [193]:
def company_sentiment_calculations(companyid):
    try:
        company_sentiments = CompanySentiment.objects.filter(company_id=companyid).values()
#        print (company_sentiments.count())
        if company_sentiments.count() >0:
            company_sentiments_list = pd.DataFrame(list(company_sentiments))
            averages = pd.Series([company_sentiments_list['sentiment_prob_very_negative'].astype(float).sum(),
                company_sentiments_list['sentiment_prob_negative'].astype(float).sum(),
                company_sentiments_list['sentiment_prob_neutral'].astype(float).sum(),
                company_sentiments_list['sentiment_prob_positive'].astype(float).sum(),
                company_sentiments_list['sentiment_prob_very_positive'].astype(float).sum()],
                index=[0,1,2,3,4])
            return (averages.idxmax(axis=1)) #pick the index with the maximum probability
        else:
            return 
    except:
        print ("sentiment error occured for company:", companyid, "error:",sys.exc_info())
        return (-1)

    
def company_zscore_calculations(companyid):
    try:
        company_zscore = CompanyAltmanZscore.objects.all().filter(company_id=companyid).values('zscore')
        r = list(company_zscore[:1])
        if r:
            return r[0]['zscore']
        return 

    except:
        print ("zscore error occured for company:", companyid, "error:",sys.exc_info())
        return (-1)

In [169]:
##get the companies from the database
companylist = pd.DataFrame(list(Company.objects.filter(~Q(marketCap=0.0)).values()))
print("total companies listed",companylist.count())
companylist['sentiment']=companylist[['id']].apply(company_sentiment_calculations, axis=1)
companylist['altman']=companylist[['id']].apply(company_zscore_calculations, axis=1)

total companies listed adrTso            5787
date_extracted    5787
exchange          5787
id                5787
industry          5787
ipoYear           5787
lastSale          5787
marketCap         5787
name              5787
sector            5787
summaryQuote      5787
symbol            5787
dtype: int64


In [213]:
companylistix = companylist.set_index(['id'])
companylistix.to_csv("companysentimentszscore.csv")